In [4]:
## import xarray as xr  ##This import is assumed for some functions
##import esmpy    ##This import is assimed for some functions

def grid_create_from_nct_rll(filename, ctk=esmpy.TypeKind.R8):
    """
    Create an ESMF grid from an NCTools regular lat-lon grid.
    Note: this function is for "demo" purpose and possibly not high quality
    Note: this function used xarray package
    :param filename: the name (possibly full path name) to an NCTools RLL grid created by make_hgrid
    :param ctk: the coordinate typekind
    :return: grid
    """

    # The NCTools RLL grid
    grid_in = xr.open_dataset(filename)

    #Suggestion from Rapahel D. work
    grid_in['lon'] = xr.DataArray(grid_in['x'].values[1::2, 1::2].copy(), dims=('ny2','nx2'))
    grid_in['lon_b'] = xr.DataArray(grid_in['x'].values[0::2,0::2].copy(), dims=('ny2p1','nx2p1'))
    grid_in['lat'] = xr.DataArray(grid_in['y'].values[1::2,1::2].copy(), dims=('ny2','nx2'))
    grid_in['lat_b'] = xr.DataArray(grid_in['y'].values[0::2,0::2].copy(), dims=('ny2p1','nx2p1'))
    grid_in['area2'] = xr.DataArray(grid_in['area'].values[1::2,1::2].copy(), dims=('ny2','nx2'))

    shape_lon = np.shape(grid_in.lon)

    grid = esmpy.Grid(np.array(shape_lon), staggerloc=[esmpy.StaggerLoc.CENTER, esmpy.StaggerLoc.CORNER],
                                coord_sys=esmpy.CoordSys.SPH_DEG)

    #Add the grid center coordinates. The default staggerloc will be the center, but we can be explicit, too.

    lons = grid.get_coords(0, staggerloc=esmpy.StaggerLoc.CENTER)
    lats = grid.get_coords(1, staggerloc=esmpy.StaggerLoc.CENTER)

    lons[...] = grid_in.lon
    lats[...] = grid_in.lat

    #Get the pointers to the corner coordinate arrays.
    cc_lats = grid.get_coords(1, staggerloc=esmpy.StaggerLoc.CORNER)
    cc_lons = grid.get_coords(0, staggerloc=esmpy.StaggerLoc.CORNER)
    #And set the corner coords
    cc_lats[...] = grid_in.lat_b
    cc_lons[...] = grid_in.lon_b

    return grid


In [5]:
def grid_create_spherical(xcoords, ycoords, xcorners, ycorners, ctk=esmpy.TypeKind.R8):
    """
    # *** THIS FUNCTION IS NOT FINISHED ***
    Create a 2 dimensional Grid using the bounds of the x and y coordiantes.
    :param xcoords: The 1st dimension or 'x' coordinates at cell centers, as a Python list or numpy Array
    :param ycoords: The 2nd dimension or 'y' coordinates at cell centers, as a Python list or numpy Array
    :param xcorners: The 1st dimension or 'x' coordinates at cell corners, as a Python list or numpy Array
    :param ycorners: The 2nd dimension or 'y' coordinates at cell corners, as a Python list or numpy Array
    :param domask: boolean to determine whether to set an arbitrary mask or not
    :param doarea: boolean to determine whether to set an arbitrary area values or not
    :param ctk: the coordinate typekind
    :return: grid
    """
    [x, y] = [0, 1]

    FORTRAN_CONTIGUOUS = True

    grid= esmpy.Grid(np.array(nam_shape), staggerloc=[esmpy.StaggerLoc.CENTER, esmpy.StaggerLoc.CORNER],
                                coord_sys=esmpy.CoordSys.SPH_DEG)

    #Add the grid center coordinates as before. The default staggerloc will be the center, but we can be explicit, too.


    cn_lon = grid_conserve.get_coords(0, staggerloc=esmpy.StaggerLoc.CENTER)
    cn_lat = grid_conserve.get_coords(1, staggerloc=esmpy.StaggerLoc.CENTER)

    cn_lon[...] = xcoords
    cn_lat[...] = ycoords

    #Get the pointers to the corner coordinate arrays.
    nc_lat = sourcegrid_conserve.get_coords(1, staggerloc=esmpy.StaggerLoc.CORNER)
    nc_lon = sourcegrid_conserve.get_coords(0, staggerloc=esmpy.StaggerLoc.CORNER)

    return grid
    

In [3]:
def grid_create_from_coordinates(xcoords, ycoords, xcorners=False, ycorners=False, corners=False, domask=False, doarea=False, ctk=esmpy.TypeKind.R8):
    """
    Create a 2 dimensional Grid using the bounds of the x and y coordiantes.
    :param xcoords: The 1st dimension or 'x' coordinates at cell centers, as a Python list or numpy Array
    :param ycoords: The 2nd dimension or 'y' coordinates at cell centers, as a Python list or numpy Array
    :param xcorners: The 1st dimension or 'x' coordinates at cell corners, as a Python list or numpy Array
    :param ycorners: The 2nd dimension or 'y' coordinates at cell corners, as a Python list or numpy Array
    :param domask: boolean to determine whether to set an arbitrary mask or not
    :param doarea: boolean to determine whether to set an arbitrary area values or not
    :param ctk: the coordinate typekind
    :return: grid
    """
    [x, y] = [0, 1]

    # create a grid given the number of grid cells in each dimension, the center stagger location is allocated, the
    # Cartesian coordinate system and type of the coordinates are specified
    max_index = np.array([len(xcoords), len(ycoords)])
    grid = esmpy.Grid(max_index, staggerloc=[esmpy.StaggerLoc.CENTER], coord_sys=esmpy.CoordSys.CART, coord_typekind=ctk)

    # set the grid coordinates using numpy arrays, parallel case is handled using grid bounds
    gridXCenter = grid.get_coords(x)
    x_par = xcoords[grid.lower_bounds[esmpy.StaggerLoc.CENTER][x]:grid.upper_bounds[esmpy.StaggerLoc.CENTER][x]]
    gridXCenter[...] = x_par.reshape((x_par.size, 1))

    gridYCenter = grid.get_coords(y)
    y_par = ycoords[grid.lower_bounds[esmpy.StaggerLoc.CENTER][y]:grid.upper_bounds[esmpy.StaggerLoc.CENTER][y]]
    gridYCenter[...] = y_par.reshape((1, y_par.size))

    # create grid corners in a slightly different manner to account for the bounds format common in CF-like files
    if corners:
        grid.add_coords([esmpy.StaggerLoc.CORNER])
        lbx = grid.lower_bounds[esmpy.StaggerLoc.CORNER][x]
        ubx = grid.upper_bounds[esmpy.StaggerLoc.CORNER][x]
        lby = grid.lower_bounds[esmpy.StaggerLoc.CORNER][y]
        uby = grid.upper_bounds[esmpy.StaggerLoc.CORNER][y]

        gridXCorner = grid.get_coords(x, staggerloc=esmpy.StaggerLoc.CORNER)
        for i0 in range(ubx - lbx - 1):
            gridXCorner[i0, :] = xcorners[i0+lbx, 0]
        gridXCorner[i0 + 1, :] = xcorners[i0+lbx, 1]

        gridYCorner = grid.get_coords(y, staggerloc=esmpy.StaggerLoc.CORNER)
        for i1 in range(uby - lby - 1):
            gridYCorner[:, i1] = ycorners[i1+lby, 0]
        gridYCorner[:, i1 + 1] = ycorners[i1+lby, 1]

    # add an arbitrary mask
    if domask:
        mask = grid.add_item(esmpy.GridItem.MASK)
        mask[:] = 1
        mask[np.where((1.75 <= gridXCenter.any() < 2.25) &
                      (1.75 <= gridYCenter.any() < 2.25))] = 0

    # add arbitrary areas values
    if doarea:
        area = grid.add_item(esmpy.GridItem.AREA)
        area[:] = 5.0

    return grid